 # Notebook para detecção de quebra de diálogo

In [1]:
from os.path import join as pathjoin;
from os import listdir
import os
import re
import json
import matplotlib.pyplot as plt
import pandas as pd;
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
import numpy as np
import sklearn.pipeline
from gensim.models import KeyedVectors
from sklearn.model_selection import GridSearchCV
import math,glob
from collections import Counter

import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm
import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Bases de desenvolvimento e teste

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
base_path = '/content/gdrive/My Drive/Mestrado/Bert/Portugues/'
dev_data_path = pathjoin(base_path,'crf_baseline/train_cut')
eval_data_path = pathjoin(base_path,'crf_baseline/test_cut')

output_data_path = pathjoin(base_path, 'experimento_dl_out/')

In [4]:
def separateEval(originPath, destinationPath, count):
    files = listdir(originPath)
    eval_files = numpy.random.choice(files,count,False)
    if not os.path.exists(destinationPath):
        os.makedirs(destinationPath)
    for file in eval_files:
        os.rename(pathjoin(originPath,file),pathjoin(destinationPath,file))

## Separando 30% em arquivos de teste

In [ ]:
# Comentando para não correr o risco de rodar novamente
#separateEval(cic_dev_path,cic_eval_path,34)
#separateEval(iris_dev_path,iris_eval_path,30)
#separateEval(tktk_dev_path,tktk_eval_path,30)
#separateEval(yi_dev_path,yi_eval_path,30)

### Funções úteis

In [5]:
def loadConversations(path):
    conversations = []
    for f in listdir(path):
        if '.log.json' in f:
            json_obj = json.load(open(pathjoin(path,f), encoding='utf-8'))
            json_obj['file_name'] = f
            conversations.append(json_obj)
    return conversations

In [6]:
# O t['class'] define o voto majoritário entre os juízes em annotations
# Geração de voto majoritário para votador humano
def SetMajorityVoteAndDistribution(conversations, threshold):
    for c in conversations:
        for t in c['turns']:
            count={}
            count['O']=0
            count['T']=0
            count['X']=0
            hasAnnotations = False
            for a in t['annotations']:
                hasAnnotations = True
                count[a['breakdown']] +=1
            del t['annotations']
            if hasAnnotations:
                #distribution
                t['O'] = count['O']*1.0 / (count['O'] + count['T'] + count['X'])
                t['T'] = count['T']*1.0 / (count['O'] + count['T'] + count['X'])
                t['X'] = count['X']*1.0 / (count['O'] + count['T'] + count['X']) 
                #majorityVoting
                t['class'] = 'O'
                t['class'] = 'X' if t['X']>=t['T'] and t['X']>=t['O'] and t['X']>=threshold else t['class']
                t['class'] = 'T' if t['T']>=t['O'] and t['T']>=t['X'] and t['T']>=threshold else t['class']
                t['class'] = 'O' if t['O']>=t['T'] and t['O']>=t['X'] and t['O']>=threshold else t['class']
                
    return conversations
        

In [7]:
def ConversationClassCount(conversations):
    count={}
    count['O']=0
    count['T']=0
    count['X']=0
    for c in conversations:
        for t in c['turns']:
            if 'class' in t:
                count[t['class']] += 1
    return count

In [8]:
def printHistogram(data,name):
    print('O = not a breakdown, T = possible breakdown, and X = breakdown')
    dataSum = ConversationClassCount(data)
    plt.hist(list(dataSum.keys()), weights=list(dataSum.values()))
    plt.title(name)
    plt.show()

In [9]:
def saveJson(outputPath, fileName, data):
    fileToSave = pathjoin(outputPath, fileName.replace('.log.json','.labels.json'))
    with open(fileToSave, 'w') as file:
        #print(data)
        file.write(json.dumps(data))

In [10]:
def outputFiles(outputPath, pipeline, test):
    test.sort_values(['file_name', 'user-index'], ascending=[True, True])
    data = {}
    data['turns'] = []
    size = test.shape[0]
    predictedValues = pipeline.predict(test)
    predictedProbas = pipeline.predict_proba(test)
    for i in range(size):
        data['dialogue-id'] = test.iloc[i]['dialogue-id']
        predictedValue = predictedValues[i]
        predictedProba = predictedProbas[i]
        
        turn ={}
        turn['turn-index'] = int(test.iloc[i]['system-index'])
        turn['labels'] = []
        
        label ={}
        label['breakdown'] = predictedValue
        label['prob-O'] = predictedProba[0]
        label['prob-T'] = predictedProba[1]
        label['prob-X'] = predictedProba[2] 
        
        turn['labels'].append(label)        
        data['turns'].append(turn)
        if  i == size-1 or test.iloc[i]['file_name'] != test.iloc[i+1]['file_name']:
            saveJson(outputPath, test.iloc[i]['file_name'], data)
            data = {}
            data['turns'] = []
            
        
#outputFiles(output_data_path,pipeline, test)

In [11]:
def scoreToLabel(score):
    score_float = float(score)
    if score_float<0.4:
        return 'O'
    elif score_float <0.8:
        return 'T'
    else:
        return 'X'

In [12]:
def resultArray(pipeline, test, testPred):
    #test.sort_values(['file_name', 'user-index'], ascending=[True, True])
    result ={}
    data = {}
    data['turns'] = []
    size = test.shape[0]
    predictedValues = pipeline.predict(testPred)
    predictedProbas = pipeline.predict_proba(testPred)
    for i in range(size):
        data['dialogue-id'] = test.iloc[i]['dialogue-id']
        result[data['dialogue-id']] = data
        predictedValue = scoreToLabel(predictedValues[i])
        predictedProba = predictedProbas[i]
        
        turn ={}
        turn['turn-index'] = int(test.iloc[i]['system-index'])
        turn['labels'] = []
        
        label ={}
        label['breakdown'] = predictedValue
        label['prob-O'] = predictedProba[0]
        label['prob-T'] = predictedProba[1]
        label['prob-X'] = predictedProba[2] 
        
        turn['labels'].append(label)        
        data['turns'].append(turn)
        if  i == size-1 or test.iloc[i]['file_name'] != test.iloc[i+1]['file_name']:
            data = {}
            data['turns'] = []
    return result

## Métodos de avaliação

In [13]:
def calc_distribution(annotations) :
    count_O = 0
    count_T = 0
    count_X = 0

    for annotation in annotations:
        if annotation['breakdown'] == 'O':
            count_O += 1
        elif annotation['breakdown'] == 'T':
            count_T += 1
        elif annotation['breakdown'] == 'X':
            count_X += 1

    prob_O = count_O*1.0 / (count_O + count_T + count_X)
    prob_T = count_T*1.0 / (count_O + count_T + count_X)
    prob_X = count_X*1.0 / (count_O + count_T + count_X)

    return [prob_O, prob_T, prob_X]

In [14]:
def majority_label(prob_O, prob_T, prob_X, threshold):

    if prob_O >= prob_T and prob_O >= prob_X and prob_O >= threshold:
        return "O"
    elif prob_T >= prob_O and prob_T >= prob_X and prob_T >= threshold:
        return "T"
    elif prob_X >= prob_T and prob_X >= prob_O and prob_X >= threshold:
        return "X"
    else:
        return "O"

In [15]:
def majority_label_lenient(prob_O, prob_T, prob_X, threshold):

    if prob_O >= prob_T + prob_X and prob_O >= threshold:
        return "O_l"
    elif prob_T + prob_X >= prob_O and prob_T + prob_X >= threshold:
        return "X_l"
    else:
        return "O_l"

In [16]:
def kld(p, q):
    k = 0.0
    for i in range(len(p)):
        if p[i] > 0:
            k += p[i] * (math.log(p[i]/q[i],2))

    return k

In [17]:
def jsd(p, q):
    m = []
    for i in range(len(p)):
        m.append((p[i]+q[i])/2.0)

    return (kld(p,m) + kld(q,m)) / 2.0

In [18]:
def mse(p, q):
    total = 0.0

    for i in range(len(p)):
        total += pow(p[i] - q[i],2)

    return total / len(p)

In [19]:
def evaluation(data_path,eval_data,threshold):

    file_num = 0
    label_num = 0

    correct_num = 0
    incorrect_num = 0

    predO_ansO = 0
    predO_ansT = 0
    predO_ansX = 0
    predT_ansO = 0
    predT_ansT = 0
    predT_ansX = 0
    predX_ansO = 0
    predX_ansT = 0
    predX_ansX = 0

    predO_ansO_l = 0
    predO_ansX_l = 0
    predT_ansO_l = 0
    predT_ansX_l = 0
    predX_ansO_l = 0
    predX_ansX_l = 0

    jsd_O_T_X_sum = 0.0
    jsd_O_TX_sum = 0.0
    jsd_OT_X_sum = 0.0
    mse_O_T_X_sum = 0.0
    mse_O_TX_sum = 0.0
    mse_OT_X_sum = 0.0

    pred_dist_list = []
    ans_dist_list = []

    #parser = argparse.ArgumentParser()
    #parser.add_argument('-p', dest='datapath', action='store', metavar='TESTDIR', required=True,
                        #help='テスト用の対話データが入ったディレクトリ')
    #parser.add_argument('-o', dest='evalpath', action='store', metavar='EVALDIR', required=True,
                        #help='評価用データが入ったディレクトリ')
    #parser.add_argument('-t', dest='threshold', action='store', metavar='THRESHOLD', required=False,
                        #help='正解ラベルを決定するためのしきい値')

    #args = parser.parse_args()

    #data_path = args.datapath
    #eval_path = args.evalpath

    #threshold = 0.0
    #if args.threshold != None:
    #    threshold = float(args.threshold)

    #if not data_path.endswith('/') or not data_path.endswith('\\'):
        #data_path += '/'
    #if not eval_path.endswith('/') or not eval_path.endswith('\\'):
        #eval_path += '/'

    data_files = glob.glob(data_path + '/*log.json')
    
    for f in data_files:
    #for data_json in data_files:
        file_num += 1

        data_fp = open(f, "r")
        data_json = json.load(data_fp)
        data_fp.close()

        dlg_id = data_json["dialogue-id"]
        #eval_fp = open(eval_path + dlg_id + ".labels.json", "r")
        #eval_json = json.load(eval_fp)
        #eval_fp.close()
        
        eval_json = eval_data[dlg_id]
        #print(dlg_id)
        eval_index = 0
        for turn in data_json['turns']:
            if turn['speaker'] == "U" or turn['annotations']==[]:  # modified Sep 17 2017
                continue

            label_num += 1

            ans_prob_dist = calc_distribution(turn['annotations'])
            ans_dist_list.append(ans_prob_dist)
            ans_label = majority_label(ans_prob_dist[0], ans_prob_dist[1], ans_prob_dist[2], threshold)

            ans_label_l = majority_label_lenient(ans_prob_dist[0], ans_prob_dist[1], ans_prob_dist[2], threshold)

            #print(f)
            target_label = eval_json['turns'][eval_index]['labels'][0]

            pred_prob_dist = [float(target_label['prob-O']), float(target_label['prob-T']), float(target_label['prob-X'])]
            pred_dist_list.append(pred_prob_dist)

            jsd_O_T_X_sum += jsd(ans_prob_dist,pred_prob_dist)
            jsd_O_TX_sum += jsd([ans_prob_dist[0],ans_prob_dist[1] + ans_prob_dist[2]],[pred_prob_dist[0],pred_prob_dist[1] + pred_prob_dist[2]])
            jsd_OT_X_sum += jsd([ans_prob_dist[0] + ans_prob_dist[1],ans_prob_dist[2]],[pred_prob_dist[0] + pred_prob_dist[1] ,pred_prob_dist[2]])

            mse_O_T_X_sum += mse(ans_prob_dist,pred_prob_dist)
            mse_O_TX_sum += mse([ans_prob_dist[0],ans_prob_dist[1] + ans_prob_dist[2]],[pred_prob_dist[0],pred_prob_dist[1] + pred_prob_dist[2]])
            mse_OT_X_sum += mse([ans_prob_dist[0] + ans_prob_dist[1],ans_prob_dist[2]],[pred_prob_dist[0] + pred_prob_dist[1] ,pred_prob_dist[2]])

            pred_label = target_label['breakdown']
            eval_index += 1

            if pred_label == ans_label:
                correct_num += 1
            else:
                incorrect_num += 1

            if pred_label == "O":
                if ans_label == "O":
                    predO_ansO += 1
                elif ans_label == "T":
                    predO_ansT += 1
                elif ans_label == "X":
                    predO_ansX += 1
            elif pred_label == "T":
                if ans_label == "O":
                    predT_ansO += 1
                elif ans_label == "T":
                    predT_ansT += 1
                elif ans_label == "X":
                    predT_ansX += 1
            elif pred_label == "X":
                if ans_label == "O":
                    predX_ansO += 1
                elif ans_label == "T":
                    predX_ansT += 1
                elif ans_label == "X":
                    predX_ansX += 1

            #lenient
            if pred_label == "O":
                if ans_label_l == "O_l":
                    predO_ansO_l += 1
                elif ans_label_l == "X_l":
                    predO_ansX_l += 1
            elif pred_label == "T":
                if ans_label_l == "O_l":
                    predT_ansO_l += 1
                elif ans_label_l == "X_l":
                    predT_ansX_l += 1
            elif pred_label == "X":
                if ans_label_l == "O_l":
                    predX_ansO_l += 1
                elif ans_label_l == "X_l":
                    predX_ansX_l += 1


#     print correct_num
#     print incorrect_num
#
#     print predO_ansO
#     print predO_ansT
#     print predO_ansX
#     print predT_ansO
#     print predT_ansT
#     print predT_ansX
#     print predX_ansO
#     print predX_ansT
#     print predX_ansX


    #print str(predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l + predO_ansX_l + predO_ansO_l)

    #print("######### Data Stats #########")
    #print("File Num : \t\t" + str(file_num))
    #print("System Utterance Num : \t" + str(label_num))
    #print("O Label Num : \t\t" + str(predO_ansO + predT_ansO + predX_ansO))
    #print("T Label Num : \t\t" + str(predO_ansT + predT_ansT + predX_ansT))
    #print("X Label Num : \t\t" + str(predO_ansX + predT_ansX + predX_ansX))
    #print("")

    #print("######### Results #########")
    #print("Accuracy : \t\t%4f" % ((correct_num*1.0) / (correct_num + incorrect_num)) + " (" + str(correct_num) + "/" + str(correct_num + incorrect_num) + ")\n")

    precision_s = 0.0
    recall_s = 0.0
    fmeasure_s = 0.0

    if predX_ansX > 0:
        if (predX_ansO + predX_ansT + predX_ansX) > 0:
            precision_s = predX_ansX*1.0 / (predX_ansO + predX_ansT + predX_ansX)
        if (predO_ansX + predT_ansX + predX_ansX) > 0:
            recall_s = predX_ansX*1.0 / (predO_ansX + predT_ansX + predX_ansX)

    if precision_s > 0 and recall_s > 0:
        fmeasure_s = (2*precision_s*recall_s)/(precision_s + recall_s)

    #print("Precision (X) : \t%4f" % (precision_s) + " (" + str(predX_ansX) + "/" + str(predX_ansO + predX_ansT + predX_ansX) + ")")
    #print("Recall    (X) : \t%4f" % (recall_s) + " (" + str(predX_ansX) + "/" + str(predO_ansX + predT_ansX + predX_ansX) + ")")
    #print("F-measure (X) : \t%4f" % (fmeasure_s) + "\n")


    precision_l = 0.0
    recall_l = 0.0
    fmeasure_l = 0.0
    if (predT_ansX_l + predX_ansX_l) > 0 and (predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l) > 0:
        precision_l = (predT_ansX_l + predX_ansX_l)*1.0 / (predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l)
    if (predT_ansX_l + predX_ansX_l) > 0 and (predO_ansX_l + predT_ansX_l + predX_ansX_l) > 0:
        recall_l = (predT_ansX_l + predX_ansX_l)*1.0 / (predO_ansX_l + predT_ansX_l + predX_ansX_l)
    if precision_l > 0 and recall_l > 0:
        fmeasure_l = (2*precision_l*recall_l)/(precision_l + recall_l)



    #print("Precision (T+X) : \t%4f" % (precision_l) + " (" + str(predT_ansX_l + predX_ansX_l) + "/" + str(predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l) + ")")
    #print("Recall    (T+X) : \t%4f" % (recall_l) + " (" + str(predT_ansX_l + predX_ansX_l) + "/" + str(predO_ansX_l + predT_ansX_l + predX_ansX_l) + ")")
    #print("F-measure (T+X) : \t%4f" % (fmeasure_l) + "\n")

    #print("JS divergence (O,T,X) : \t%4f" % (jsd_O_T_X_sum / (correct_num + incorrect_num)))
    #print("JS divergence (O,T+X) : \t%4f" % (jsd_O_TX_sum / (correct_num + incorrect_num)))
    #print("JS divergence (O+T,X) : \t%4f" % (jsd_OT_X_sum / (correct_num + incorrect_num)) + "\n")

    #print("Mean squared error (O,T,X) : \t%4f" % (mse_O_T_X_sum / (correct_num + incorrect_num)))
    #print("Mean squared error (O,T+X) : \t%4f" % (mse_O_TX_sum / (correct_num + incorrect_num)))
    #print("Mean squared error (O+T,X) : \t%4f" % (mse_OT_X_sum / (correct_num + incorrect_num)))
    #print("###########################")

In [20]:
def evaluationAsDict(data_path,eval_data,threshold):

    file_num = 0
    label_num = 0

    correct_num = 0
    incorrect_num = 0

    predO_ansO = 0
    predO_ansT = 0
    predO_ansX = 0
    predT_ansO = 0
    predT_ansT = 0
    predT_ansX = 0
    predX_ansO = 0
    predX_ansT = 0
    predX_ansX = 0

    predO_ansO_l = 0
    predO_ansX_l = 0
    predT_ansO_l = 0
    predT_ansX_l = 0
    predX_ansO_l = 0
    predX_ansX_l = 0

    jsd_O_T_X_sum = 0.0
    jsd_O_TX_sum = 0.0
    jsd_OT_X_sum = 0.0
    mse_O_T_X_sum = 0.0
    mse_O_TX_sum = 0.0
    mse_OT_X_sum = 0.0

    pred_dist_list = []
    ans_dist_list = []

    #parser = argparse.ArgumentParser()
    #parser.add_argument('-p', dest='datapath', action='store', metavar='TESTDIR', required=True,
                        #help='テスト用の対話データが入ったディレクトリ')
    #parser.add_argument('-o', dest='evalpath', action='store', metavar='EVALDIR', required=True,
                        #help='評価用データが入ったディレクトリ')
    #parser.add_argument('-t', dest='threshold', action='store', metavar='THRESHOLD', required=False,
                        #help='正解ラベルを決定するためのしきい値')

    #args = parser.parse_args()

    #data_path = args.datapath
    #eval_path = args.evalpath

    #threshold = 0.0
    #if args.threshold != None:
    #    threshold = float(args.threshold)

    #if not data_path.endswith('/') or not data_path.endswith('\\'):
        #data_path += '/'
    #if not eval_path.endswith('/') or not eval_path.endswith('\\'):
        #eval_path += '/'

    data_files = glob.glob(data_path + '/*log.json')
    
    for f in data_files:
    #for data_json in data_files:
        file_num += 1

        data_fp = open(f, "r")
        data_json = json.load(data_fp)
        data_fp.close()

        dlg_id = data_json["dialogue-id"]
        #eval_fp = open(eval_path + dlg_id + ".labels.json", "r")
        #eval_json = json.load(eval_fp)
        #eval_fp.close()
        
        eval_json = eval_data[dlg_id]
        #print(dlg_id)
        eval_index = 0
        #print(len(data_json['turns']))
        #print(len(eval_json['turns']))
        for turn in data_json['turns']:
            if turn['speaker'] == "U" or turn['annotations']==[]:  # modified Sep 17 2017
                continue

            label_num += 1

            ans_prob_dist = calc_distribution(turn['annotations'])
            ans_dist_list.append(ans_prob_dist)
            ans_label = majority_label(ans_prob_dist[0], ans_prob_dist[1], ans_prob_dist[2], threshold)

            ans_label_l = majority_label_lenient(ans_prob_dist[0], ans_prob_dist[1], ans_prob_dist[2], threshold)

            #print(turn)
            #print(eval_json)
            #print(eval_index)
            target_label = eval_json['turns'][eval_index]['labels'][0]
            
            #print(str(ans_label)+','+str(target_label['breakdown'])+','+ str(turn['turn-index'])+','+ str(eval_json['turns'][eval_index]['turn-index']))

            pred_prob_dist = [float(target_label['prob-O']), float(target_label['prob-T']), float(target_label['prob-X'])]
            pred_dist_list.append(pred_prob_dist)

            jsd_O_T_X_sum += jsd(ans_prob_dist,pred_prob_dist)
            jsd_O_TX_sum += jsd([ans_prob_dist[0],ans_prob_dist[1] + ans_prob_dist[2]],[pred_prob_dist[0],pred_prob_dist[1] + pred_prob_dist[2]])
            jsd_OT_X_sum += jsd([ans_prob_dist[0] + ans_prob_dist[1],ans_prob_dist[2]],[pred_prob_dist[0] + pred_prob_dist[1] ,pred_prob_dist[2]])

            mse_O_T_X_sum += mse(ans_prob_dist,pred_prob_dist)
            mse_O_TX_sum += mse([ans_prob_dist[0],ans_prob_dist[1] + ans_prob_dist[2]],[pred_prob_dist[0],pred_prob_dist[1] + pred_prob_dist[2]])
            mse_OT_X_sum += mse([ans_prob_dist[0] + ans_prob_dist[1],ans_prob_dist[2]],[pred_prob_dist[0] + pred_prob_dist[1] ,pred_prob_dist[2]])

            pred_label = target_label['breakdown']
            eval_index += 1

            if pred_label == ans_label:
                correct_num += 1
            else:
                incorrect_num += 1

            if pred_label == "O":
                if ans_label == "O":
                    predO_ansO += 1
                elif ans_label == "T":
                    predO_ansT += 1
                elif ans_label == "X":
                    predO_ansX += 1
            elif pred_label == "T":
                if ans_label == "O":
                    predT_ansO += 1
                elif ans_label == "T":
                    predT_ansT += 1
                elif ans_label == "X":
                    predT_ansX += 1
            elif pred_label == "X":
                if ans_label == "O":
                    predX_ansO += 1
                elif ans_label == "T":
                    predX_ansT += 1
                elif ans_label == "X":
                    predX_ansX += 1

            #lenient
            if pred_label == "O":
                if ans_label_l == "O_l":
                    predO_ansO_l += 1
                elif ans_label_l == "X_l":
                    predO_ansX_l += 1
            elif pred_label == "T":
                if ans_label_l == "O_l":
                    predT_ansO_l += 1
                elif ans_label_l == "X_l":
                    predT_ansX_l += 1
            elif pred_label == "X":
                if ans_label_l == "O_l":
                    predX_ansO_l += 1
                elif ans_label_l == "X_l":
                    predX_ansX_l += 1


#     print correct_num
#     print incorrect_num
#
#     print predO_ansO
#     print predO_ansT
#     print predO_ansX
#     print predT_ansO
#     print predT_ansT
#     print predT_ansX
#     print predX_ansO
#     print predX_ansT
#     print predX_ansX


    #print str(predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l + predO_ansX_l + predO_ansO_l)
    
    print("######### Data Stats #########")
    print("File Num : \t\t" + str(file_num))
    print("System Utterance Num : \t" + str(label_num))
    print("O Label Num : \t\t" + str(predO_ansO + predT_ansO + predX_ansO))
    print("T Label Num : \t\t" + str(predO_ansT + predT_ansT + predX_ansT))
    print("X Label Num : \t\t" + str(predO_ansX + predT_ansX + predX_ansX))
    print("")
    print("O Pred Num : \t\t" + str(predO_ansO + predO_ansT + predO_ansX))
    print("T Pred Num : \t\t" + str(predT_ansO + predT_ansT + predT_ansX))
    print("X Pred Num : \t\t" + str(predX_ansO + predX_ansT + predX_ansX))

    print("######### Results #########")
    print("Accuracy : \t\t%4f" % ((correct_num*1.0) / (correct_num + incorrect_num)) + " (" + str(correct_num) + "/" + str(correct_num + incorrect_num) + ")\n")

    precision_s = 0.0
    recall_s = 0.0
    fmeasure_s = 0.0

    if predX_ansX > 0:
        if (predX_ansO + predX_ansT + predX_ansX) > 0:
            precision_s = predX_ansX*1.0 / (predX_ansO + predX_ansT + predX_ansX)
        if (predO_ansX + predT_ansX + predX_ansX) > 0:
            recall_s = predX_ansX*1.0 / (predO_ansX + predT_ansX + predX_ansX)

    if precision_s > 0 and recall_s > 0:
        fmeasure_s = (2*precision_s*recall_s)/(precision_s + recall_s)

    print("Precision (X) : \t%4f" % (precision_s) + " (" + str(predX_ansX) + "/" + str(predX_ansO + predX_ansT + predX_ansX) + ")")
    print("Recall    (X) : \t%4f" % (recall_s) + " (" + str(predX_ansX) + "/" + str(predO_ansX + predT_ansX + predX_ansX) + ")")
    print("F-measure (X) : \t%4f" % (fmeasure_s) + "\n")


    precision_l = 0.0
    recall_l = 0.0
    fmeasure_l = 0.0
    if (predT_ansX_l + predX_ansX_l) > 0 and (predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l) > 0:
        precision_l = (predT_ansX_l + predX_ansX_l)*1.0 / (predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l)
    if (predT_ansX_l + predX_ansX_l) > 0 and (predO_ansX_l + predT_ansX_l + predX_ansX_l) > 0:
        recall_l = (predT_ansX_l + predX_ansX_l)*1.0 / (predO_ansX_l + predT_ansX_l + predX_ansX_l)
    if precision_l > 0 and recall_l > 0:
        fmeasure_l = (2*precision_l*recall_l)/(precision_l + recall_l)



    print("Precision (T+X) : \t%4f" % (precision_l) + " (" + str(predT_ansX_l + predX_ansX_l) + "/" + str(predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l) + ")")
    print("Recall    (T+X) : \t%4f" % (recall_l) + " (" + str(predT_ansX_l + predX_ansX_l) + "/" + str(predO_ansX_l + predT_ansX_l + predX_ansX_l) + ")")
    print("F-measure (T+X) : \t%4f" % (fmeasure_l) + "\n")

    print("JS divergence (O,T,X) : \t%4f" % (jsd_O_T_X_sum / (correct_num + incorrect_num)))
    print("JS divergence (O,T+X) : \t%4f" % (jsd_O_TX_sum / (correct_num + incorrect_num)))
    print("JS divergence (O+T,X) : \t%4f" % (jsd_OT_X_sum / (correct_num + incorrect_num)) + "\n")

    print("Mean squared error (O,T,X) : \t%4f" % (mse_O_T_X_sum / (correct_num + incorrect_num)))
    print("Mean squared error (O,T+X) : \t%4f" % (mse_O_TX_sum / (correct_num + incorrect_num)))
    print("Mean squared error (O+T,X) : \t%4f" % (mse_OT_X_sum / (correct_num + incorrect_num)))
    print("###########################")
    
    result_ans = {
        "fileNumber":file_num,
        "systemUtterance":label_num,
        "ansO": predO_ansO + predT_ansO + predX_ansO,
        "ansT": predO_ansT + predT_ansT + predX_ansT,
        "ansX": predO_ansX + predT_ansX + predX_ansX,
        "accuracy": ((correct_num*1.0) / (correct_num + incorrect_num)),
        "precisionX": precision_s,
        "predX_ansX":predX_ansX,
        "total_predX": predX_ansO + predX_ansT + predX_ansX,
        "total_ansX": predO_ansX + predT_ansX + predX_ansX,
        "recallX": recall_s,
        "f1X": fmeasure_s,
        
        "precisionTX":precision_l,
        "recallTX": recall_l,
        "f1TX": fmeasure_l,
        
        "totalTXAnsX": predT_ansX_l + predX_ansX_l,
        "totalPredX+PredT" : predX_ansO_l + predX_ansX_l + predT_ansO_l + predT_ansX_l,
        "totalAnsX" : predO_ansX_l + predT_ansX_l + predX_ansX_l,
        
        "js-O-T-X" : (jsd_O_T_X_sum / (correct_num + incorrect_num)),
        "js-O-T+X" : (jsd_O_TX_sum / (correct_num + incorrect_num)),
        "js-O+T-X" : (jsd_OT_X_sum / (correct_num + incorrect_num)),

        "mse-O-T-X" :  (mse_O_T_X_sum / (correct_num + incorrect_num)),
        "mse-O-T+X" :  (mse_O_TX_sum / (correct_num + incorrect_num)),
        "mse-O+T-X" :  (mse_OT_X_sum / (correct_num + incorrect_num))        
        
    }
    
    return result_ans

## Computando distribuição e voto majoritário

In [21]:
json_dev_dev = SetMajorityVoteAndDistribution(loadConversations(dev_data_path),0.5)
json_dev_test = SetMajorityVoteAndDistribution(loadConversations(eval_data_path),0.5)

## Montando o conjunto

In [23]:
def PrepareDataSet(conversations):
    items = conversations.copy()
    result = []
    for c in items:
        #if c['turns'][0]['speaker']=='S':
            #del c['turns'][0]
        turns_len = len(c['turns'])
        userTurnTail = ''
        systemTurnTail = ''
        for i in range(0,turns_len):
            if i == 0 and c['turns'][i]['speaker'] == 'S':
                line = {}
                line['file_name'] = c['file_name']
                line['dialogue-id'] = c['dialogue-id']
                line['user-index'] = c['turns'][i]['turn-index']
                line['user-utterance'] = ''
                line['system-utterance'] = c['turns'][i]['utterance']                
                #line['user-utterance-tail'] = ''
                #line['system-utterance-tail'] = c['turns'][i]['utterance']                
                line['system-index'] = c['turns'][i]['turn-index']                
                line['O'] = 1
                line['T'] = 0
                line['X'] = 0
                line['class']=0
                result.append(line)
                
            if i+1<turns_len and c['turns'][i]['speaker'] == 'U' and c['turns'][i+1]['speaker'] == 'S':
                line = {}
                line['file_name'] = c['file_name']
                line['dialogue-id'] = c['dialogue-id']
                line['user-index'] = c['turns'][i]['turn-index']
                
                #userTurnTail = userTurnTail + ' ' + c['turns'][i]['utterance']
                line['user-utterance'] = c['turns'][i]['utterance']
                
                #systemTurnTail = systemTurnTail + ' ' + c['turns'][i+1]['utterance']
                line['system-utterance'] = c['turns'][i+1]['utterance']
                
                #line['user-utterance-tail'] = GetLastNTurns(c['turns'],i,windowSize)
                #line['system-utterance-tail'] = GetLastNTurns(c['turns'],i+1,windowSize)
                
                line['system-index'] = c['turns'][i+1]['turn-index']                
                line['O'] = c['turns'][i+1]['O']
                line['T'] = c['turns'][i+1]['T']
                line['X'] = c['turns'][i+1]['X']
                if c['turns'][i+1]['class'] == 'O':
                    line['class']=0
                elif c['turns'][i+1]['class'] == 'T':
                    line['class'] = 0.5
                elif c['turns'][i+1]['class'] == 'X':
                    line['class'] = 1
                else:
                    line['class'] = None
                #line['class'] = c['turns'][i+1]['class']
                result.append(line)
    return result

In [24]:
PrepareDataSet(json_dev_dev)[:2]

[{'O': 1,
  'T': 0,
  'X': 0,
  'class': 0,
  'dialogue-id': '0014b274-28c5-4ca0-8ed9-b5ecd3340364',
  'file_name': '0014b274-28c5-4ca0-8ed9-b5ecd3340364.log.json',
  'system-index': 1,
  'system-utterance': 'Olá, MATHEUS+HENRIQUE+MORETTI+DE+OLIVEIRA! Sou o assistente virtual da Clear.       Seja bem-vindo(a)! Em que posso te ajudar?',
  'user-index': 1,
  'user-utterance': ''},
 {'O': 1.0,
  'T': 0.0,
  'X': 0.0,
  'class': 0,
  'dialogue-id': '0014b274-28c5-4ca0-8ed9-b5ecd3340364',
  'file_name': '0014b274-28c5-4ca0-8ed9-b5ecd3340364.log.json',
  'system-index': 3,
  'system-utterance': 'Bom dia! Como eu posso te ajudar? ',
  'user-index': 2,
  'user-utterance': 'Bom dia'}]

## Modelagem

In [25]:
 !wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip
 !unzip download*.zip

--2021-01-08 03:07:05--  http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348388454 (332M) [application/octet-stream]
Saving to: ‘download.php?file=embeddings%2Fglove%2Fglove_s100.zip’

download.php?file=e 100%[===================>] 332.25M  2.69MB/s    in 2m 12s  

2021-01-08 03:09:18 (2.51 MB/s) - ‘download.php?file=embeddings%2Fglove%2Fglove_s100.zip’ saved [348388454/348388454]

Archive:  download.php?file=embeddings%2Fglove%2Fglove_s100.zip
  inflating: glove_s100.txt          


In [26]:
EMBEDDING_DIM = 100
MAX_LENGTH = 120
MAX_LEN_DIALOGUE = 10
BATCH_SIZE = 10
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'

In [27]:
stopwords = [ "a","acerca","adeus","agora","ainda","alem","algmas","algo","algumas","alguns","ali","além","ambas","ambos","ano","anos","antes","ao","aonde","aos","apenas","apoio","apontar","apos","após","aquela","aquelas","aquele","aqueles","aqui","aquilo","as","assim","através","atrás","até","aí","baixo","bastante","bem","boa","boas","bom","bons","breve","cada","caminho","catorze","cedo","cento","certamente","certeza","cima","cinco","coisa","com","como","comprido","conhecido","conselho","contra","contudo","corrente","cuja","cujas","cujo","cujos","custa","cá","da","daquela","daquelas","daquele","daqueles","dar","das","de","debaixo","dela","delas","dele","deles","demais","dentro","depois","desde","desligado","dessa","dessas","desse","desses","desta","destas","deste","destes","deve","devem","deverá","dez","dezanove","dezasseis","dezassete","dezoito","dia","diante","direita","dispoe","dispoem","diversa","diversas","diversos","diz","dizem","dizer","do","dois","dos","doze","duas","durante","dá","dão","dúvida","e","ela","elas","ele","eles","em","embora","enquanto","entao","entre","então","era","eram","essa","essas","esse","esses","esta","estado","estamos","estar","estará","estas","estava","estavam","este","esteja","estejam","estejamos","estes","esteve","estive","estivemos","estiver","estivera","estiveram","estiverem","estivermos","estivesse","estivessem","estiveste","estivestes","estivéramos","estivéssemos","estou","está","estás","estávamos","estão","eu","exemplo","falta","fará","favor","faz","fazeis","fazem","fazemos","fazer","fazes","fazia","faço","fez","fim","final","foi","fomos","for","fora","foram","forem","forma","formos","fosse","fossem","foste","fostes","fui","fôramos","fôssemos","geral","grande","grandes","grupo","ha","haja","hajam","hajamos","havemos","havia","hei","hoje","hora","horas","houve","houvemos","houver","houvera","houveram","houverei","houverem","houveremos","houveria","houveriam","houvermos","houverá","houverão","houveríamos","houvesse","houvessem","houvéramos","houvéssemos","há","hão","iniciar","inicio","ir","irá","isso","ista","iste","isto","já","lado","lhe","lhes","ligado","local","logo","longe","lugar","lá","maior","maioria","maiorias","mais","mal","mas","me","mediante","meio","menor","menos","meses","mesma","mesmas","mesmo","mesmos","meu","meus","mil","minha","minhas","momento","muito","muitos","máximo","mês","na","nada","nao","naquela","naquelas","naquele","naqueles","nas","nem","nenhuma","nessa","nessas","nesse","nesses","nesta","nestas","neste","nestes","no","noite","nome","nos","nossa","nossas","nosso","nossos","nova","novas","nove","novo","novos","num","numa","numas","nunca","nuns","não","nível","nós","número","o","obra","obrigada","obrigado","oitava","oitavo","oito","onde","ontem","onze","os","ou","outra","outras","outro","outros","para","parece","parte","partir","paucas","pegar","pela","pelas","pelo","pelos","perante","perto","pessoas","pode","podem","poder","poderá","podia","pois","ponto","pontos","por","porque","porquê","portanto","posição","possivelmente","posso","possível","pouca","pouco","poucos","povo","primeira","primeiras","primeiro","primeiros","promeiro","propios","proprio","própria","próprias","próprio","próprios","próxima","próximas","próximo","próximos","puderam","pôde","põe","põem","quais","qual","qualquer","quando","quanto","quarta","quarto","quatro","que","quem","quer","quereis","querem","queremas","queres","quero","questão","quieto","quinta","quinto","quinze","quáis","quê","relação","sabe","sabem","saber","se","segunda","segundo","sei","seis","seja","sejam","sejamos","sem","sempre","sendo","ser","serei","seremos","seria","seriam","será","serão","seríamos","sete","seu","seus","sexta","sexto","sim","sistema","sob","sobre","sois","somente","somos","sou","sua","suas","são","sétima","sétimo","só","tal","talvez","tambem","também","tanta","tantas","tanto","tarde","te","tem","temos","tempo","tendes","tenha","tenham","tenhamos","tenho","tens","tentar","tentaram","tente","tentei","ter","terceira","terceiro","terei","teremos","teria","teriam","terá","terão","teríamos","teu","teus","teve","tinha","tinham","tipo","tive","tivemos","tiver","tivera","tiveram","tiverem","tivermos","tivesse","tivessem","tiveste","tivestes","tivéramos","tivéssemos","toda","todas","todo","todos","trabalhar","trabalho","treze","três","tu","tua","tuas","tudo","tão","tém","têm","tínhamos","um","uma","umas","uns","usa","usar","vai","vais","valor","veja","vem","vens","ver","verdade","verdadeiro","vez","vezes","viagem","vindo","vinte","você","vocês","vos","vossa","vossas","vosso","vossos","vários","vão","vêm","vós","zero","à","às","área","é","éramos","és","último" ]

def preprocess_text(text):
    text = text.lower()
    cleaned = re.sub(r"[^'\w\s]", " ", text)
    cleaned = re.sub(r"\s\s", " ", cleaned).strip()
    cleaned = " ".join([w for w in cleaned.split() if w not in stopwords])
    return cleaned

In [28]:
embeddings_index = {}
with open(f'glove_s{EMBEDDING_DIM}.txt', 'r', encoding="utf-8") as f:
  for line in f.readlines()[1:]:
      values = line.split()
      word = values[0]
      try:
        coefs = np.asarray(values[1:], dtype='float32')
        if coefs.shape[0]==EMBEDDING_DIM:
          embeddings_index[word] = coefs
      except:
        print(word)

# from gensim.models import KeyedVectors
# word2vecGoogleNewsVectors = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
# w2v = KeyedVectors.load_word2vec_format(word2vecGoogleNewsVectors, binary=True)
# embeddings_index = {}
# for w in w2v.vocab.keys():
#   embeddings_index[w] = w2v.get_vector(w)
# del w2v
# del word2vecGoogleNewsVectors

r$
00
三藏法師玄奘奉
r$


In [29]:
label_mapping = {
    'O': 0,
    'T': 1,
    'X': 2,
    'extra': 3
}
reverse_mapping = {v:k for k, v in label_mapping.items()}

train_utterances = []
train_features = []
train_labels = []
cur_dialogue = '0014b274-28c5-4ca0-8ed9-b5ecd3340364'
dialogue = []
current_dialogue_labels = []
new_features = []
for info in tqdm(PrepareDataSet(json_dev_dev)):
  flag = 0
  if info['dialogue-id']!=cur_dialogue or len(dialogue)>=MAX_LEN_DIALOGUE:
    flag = 1
    new_index = index
    while len(dialogue)<MAX_LEN_DIALOGUE:
      dialogue.append([np.zeros((MAX_LENGTH, EMBEDDING_DIM)), np.zeros((MAX_LENGTH, EMBEDDING_DIM))])
      current_dialogue_labels.append([0.0, 0.0, 0.0])
      new_index += 1
      new_features.append([0.0, new_index])
    train_utterances.append(dialogue)
    train_labels.append(current_dialogue_labels)
    train_features.append(new_features)
    dialogue = []
    current_dialogue_labels = []
    new_features = []
    cur_dialogue = info['dialogue-id']
  user = preprocess_text(info['user-utterance'])
  user_tokens = tokenize.word_tokenize(user, language="portuguese")
  user = user_tokens[:MAX_LENGTH]+["<oov>"]*(MAX_LENGTH-len(user_tokens))
  user = [embeddings_index.get(w, np.zeros(EMBEDDING_DIM)) for w in user]
  system = preprocess_text(info['system-utterance'])
  system_tokens = tokenize.word_tokenize(system, language="portuguese")
  system = system_tokens[:MAX_LENGTH]+["<oov>"]*(MAX_LENGTH-len(system_tokens))
  system = [embeddings_index.get(w, np.zeros(EMBEDDING_DIM)) for w in system]
  mean_len = (len(system_tokens)+len(user_tokens))/2.0
  index = (info["user-index"]+info["system-index"])/2.0
  new_features.append([mean_len, index])
  dialogue.append([user, system])
  current_dialogue_labels.append([info['O'], info['T'], info['X']])

if flag==0:
  new_index = index
  while len(dialogue)<MAX_LEN_DIALOGUE:
    dialogue.append([np.zeros((MAX_LENGTH, EMBEDDING_DIM)), np.zeros((MAX_LENGTH, EMBEDDING_DIM))])
    current_dialogue_labels.append([0.0, 0.0, 0.0])
    new_index +=1
    new_features.append([0.0, new_index])
  train_utterances.append(dialogue)
  train_labels.append(current_dialogue_labels)
  train_features.append(new_features)
  dialogue = []
  current_dialogue_labels = []
  cur_dialogue = info['dialogue-id']

test_utterances = []
test_features = []
test_labels = []
cur_dialogue = '084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d'
dialogue = []
current_dialogue_labels = []
new_features = []
for info in tqdm(PrepareDataSet(json_dev_test)):
  flag = 0
  if info['dialogue-id']!=cur_dialogue  or len(dialogue)>=MAX_LEN_DIALOGUE:
    flag = 1
    new_index = index
    while len(dialogue)<MAX_LEN_DIALOGUE:
      dialogue.append([np.zeros((MAX_LENGTH, EMBEDDING_DIM)), np.zeros((MAX_LENGTH, EMBEDDING_DIM))])
      current_dialogue_labels.append([0.0, 0.0, 0.0])
      new_index += 1
      new_features.append([0.0, new_index])
    test_utterances.append(dialogue)
    test_labels.append(current_dialogue_labels)
    test_features.append(new_features)
    dialogue = []
    current_dialogue_labels = []
    new_features = []
    cur_dialogue = info['dialogue-id']
  user = preprocess_text(info['user-utterance'])
  user_tokens = tokenize.word_tokenize(user, language="portuguese")
  user = user_tokens[:MAX_LENGTH]+["<oov>"]*(MAX_LENGTH-len(user_tokens))
  user = [embeddings_index.get(w, np.zeros(EMBEDDING_DIM)) for w in user]
  system = preprocess_text(info['system-utterance'])
  system_tokens = tokenize.word_tokenize(system, language="portuguese")
  system = system_tokens[:MAX_LENGTH]+["<oov>"]*(MAX_LENGTH-len(system_tokens))
  system = [embeddings_index.get(w, np.zeros(EMBEDDING_DIM)) for w in system]
  mean_len = (len(system_tokens)+len(user_tokens))/2.0
  index = (info["user-index"]+info["system-index"])/2.0
  new_features.append([mean_len, index])
  dialogue.append([user, system])
  current_dialogue_labels.append([info['O'], info['T'], info['X']])

if flag==0:
  new_index = index
  while len(dialogue)<MAX_LEN_DIALOGUE:
    dialogue.append([np.zeros((MAX_LENGTH, EMBEDDING_DIM)), np.zeros((MAX_LENGTH, EMBEDDING_DIM))])
    current_dialogue_labels.append([0.0, 0.0, 0.0])
    new_index += 1
    new_features.append([0.0, new_index])
  test_utterances.append(dialogue)
  test_labels.append(current_dialogue_labels)
  test_features.append(new_features)
  dialogue = []
  current_dialogue_labels = []
  cur_dialogue = info['dialogue-id']

train_features = np.array(train_features, dtype="float32")
train_features = train_features/np.max(train_features, axis=0)
train_utterances = np.array(train_utterances)
train_labels = np.array(train_labels)
test_utterances = np.array(test_utterances)
test_features = np.array(test_features)
test_features = test_features/np.max(train_features, axis=0)
test_labels = np.array(test_labels)

100%|██████████| 11745/11745 [00:10<00:00, 1091.26it/s]


In [30]:
train_utterances.shape

(1278, 10, 2, 120, 100)

In [31]:
train_labels.shape

(1278, 10, 3)

In [32]:
class WeightedCategoricalCrossEntropy(tf.keras.losses.Loss):

  def __init__(self, class_distribution):
    super().__init__()
    weights = class_distribution.sum()-class_distribution
    self.weights = weights/np.sum(weights)

  def __call__(self, y_true, y_pred, **kwargs):
    mask = self.weights*y_true*K.log(K.clip(y_pred, K.epsilon(), 1.0))
    example_loss = K.sum(mask, axis=1)
    return -K.mean(example_loss)
class_distribution = np.sum(np.sum(train_labels, axis=0), axis=0)
class_distribution

array([6809.,    0., 1354.])

In [33]:
input = tf.keras.layers.Input(shape=(MAX_LEN_DIALOGUE, 2, MAX_LENGTH, EMBEDDING_DIM))
x = tf.keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=-2))(input)
x = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-2))(x)
for n in [64, 32, 16]:
  x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(n, return_sequences=True))(x)
  x = tf.keras.layers.Dropout(0.2)(x)
input2 = tf.keras.layers.Input(shape=(MAX_LEN_DIALOGUE, 2))
x = tf.keras.layers.Dense(16, activation="relu")(x)
x = tf.keras.layers.concatenate([x, input2], axis=-1)
x = tf.keras.layers.Dense(8, activation="relu")(x)
output = tf.keras.layers.Dense(3, activation="softmax")(x)
mymodel = tf.keras.Model(inputs=[input, input2], outputs=output)

mymodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="categorical_crossentropy")
mymodel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 2, 120,  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 10, 2, 100)   0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 10, 100)      0           lambda[0][0]                     
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 10, 128)      63744       lambda_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# tf.keras.utils.plot_model(mymodel, show_shapes=True)

In [34]:
early = tf.keras.callbacks.EarlyStopping(patience=5)
best = tf.keras.callbacks.ModelCheckpoint("model_leo_03.h5", save_best_only=True, save_weights_only=True)
mymodel.fit(x=[train_utterances, train_features],
            y=train_labels,
            # validation_data=([test_utterances, test_features], test_labels),
            # validation_batch_size=10,
            validation_split=0.1,
            callbacks=[early, best],
            epochs=100, 
            batch_size=BATCH_SIZE
            )

Epoch 1/100
115/115 [==============================] - 14s 35ms/step - loss: 0.4494 - val_loss: 0.2742
Epoch 2/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2642 - val_loss: 0.2342
Epoch 3/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2309 - val_loss: 0.2239
Epoch 4/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2166 - val_loss: 0.2169
Epoch 5/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2202 - val_loss: 0.2038
Epoch 6/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2080 - val_loss: 0.2086
Epoch 7/100
115/115 [==============================] - 2s 16ms/step - loss: 0.2019 - val_loss: 0.2076
Epoch 8/100
115/115 [==============================] - 2s 16ms/step - loss: 0.1960 - val_loss: 0.1966
Epoch 9/100
115/115 [==============================] - 2s 16ms/step - loss: 0.1903 - val_loss: 0.1958
Epoch 10/100
115/115 [==============================] - 2s 16ms/step - loss: 0.19

In [35]:
def resultArray(model, test, testPred):
    result ={}
    data = {}
    data['turns'] = []
    size = len(test)
    predictedProbas = model.predict(testPred).reshape(-1,3)
    predictedValues = np.argmax(predictedProbas, axis=-1)
    for i in range(size):
        data['dialogue-id'] = test[i]['dialogue-id']
        result[data['dialogue-id']] = data
        predictedValue = reverse_mapping[predictedValues[i]]
        predictedProba = predictedProbas[i]
        
        turn ={}
        turn['turn-index'] = int(test[i]['system-index'])
        turn['labels'] = []
        
        label ={}
        label['breakdown'] = predictedValue
        label['prob-O'] = predictedProba[0]
        label['prob-T'] = predictedProba[1]
        label['prob-X'] = predictedProba[2] 
        
        turn['labels'].append(label)        
        data['turns'].append(turn)
        
        if  i == size-1 or test[i]['file_name'] != test[i+1]['file_name']:
            data = {}
            data['turns'] = []
    return result

In [36]:
final_model = mymodel
final_model.load_weights("/content/gdrive/My Drive/Mestrado/Word2Vec/model_port.h5")

predicted = resultArray(final_model, PrepareDataSet(json_dev_test), [test_utterances, test_features])
output = evaluationAsDict(eval_data_path, predicted, 0.5)

######### Data Stats #########
File Num : 		1387
System Utterance Num : 	11732
O Label Num : 		10072
T Label Num : 		0
X Label Num : 		1660

O Pred Num : 		4983
T Pred Num : 		0
X Pred Num : 		6749
######### Results #########
Accuracy : 		0.450648 (5287/11732)

Precision (X) : 	0.145503 (982/6749)
Recall    (X) : 	0.591566 (982/1660)
F-measure (X) : 	0.233559

Precision (T+X) : 	0.145503 (982/6749)
Recall    (T+X) : 	0.591566 (982/1660)
F-measure (T+X) : 	0.233559

JS divergence (O,T,X) : 	0.483908
JS divergence (O,T+X) : 	0.483647
JS divergence (O+T,X) : 	0.475938

Mean squared error (O,T,X) : 	0.324048
Mean squared error (O,T+X) : 	0.489162
Mean squared error (O+T,X) : 	0.482724
###########################


In [37]:
dict = {}
for p in predicted:
    for t in predicted[p]['turns']:
        dict[p + '-' + str(t['turn-index'])]= t['labels'][0]['breakdown']
json.dumps(dict)

'{"084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-2": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-4": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-6": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-8": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-10": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-12": "O", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-14": "X", "084bd95c-cfd9-4bd3-8d2f-a7ce0ff1147d-16": "O", "00094a5b-939c-4f6e-8548-c0c3c375e2bd-2": "X", "00094a5b-939c-4f6e-8548-c0c3c375e2bd-4": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-2": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-4": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-6": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-8": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-10": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-12": "X", "00102824-a5ec-44f4-b0af-4ae49b53fc96-14": "X", "08819310-eeb8-4637-ab1c-7438da284a76-2": "X", "08819310-eeb8-4637-ab1c-7438da284a76-4": "X", "08819310-eeb8-4637-ab1c-7438da284a76-6": "X", "08819310-eeb8-4637-ab1c-7438da284a76-8": "O", "088